# Natural Language Processing Lab

In this lab we will further explore Scikit's and NLTK's capabilities to process text. We will use the 20 Newsgroup dataset, which is provided by Scikit-Learn.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:

categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=('headers', 'footers', 'quotes'))

## 1. Data inspection

We have downloaded a few newsgroup categories and removed headers, footers and quotes.

Let's inspect them.

1. What data type is `data_train`
> sklearn.datasets.base.Bunch
- Is it like a list? Or like a Dictionary? or what?
> Dict
- How many data points does it contain?
- Inspect the first data point, what does it look like?
> A blurb of text

In [4]:
print(type(data_train))
print(len(data_train['data']))
print(data_train['data'][0])

<class 'sklearn.datasets.base.Bunch'>
2034
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych


## 2. Bag of Words model

Let's train a model using a simple count vectorizer

1. Initialize a standard CountVectorizer and fit the training data
- how big is the feature dictionary
- repeat eliminating english stop words
- is the dictionary smaller?
- transform the training data using the trained vectorizer
- what are the 20 words that are most common in the whole corpus?
- what are the 20 most common words in each of the 4 classes?
- evaluate the performance of a Lotistic Regression on the features extracted by the CountVectorizer
    - you will have to transform the test_set too. Be carefule to use the trained vectorizer, without re-fitting it
- try the following 3 modification:
    - restrict the max_features
    - change max_df and min_df
    - use a fixed vocabulary of size 80 combining the 20 most common words per group found earlier
- for each of the above print a confusion matrix and investigate what gets mixed
> Anwer: not surprisingly if we reduce the feature space we lose accuracy
- print out the number of features for each model

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
countvect = CountVectorizer()
countvect.fit(data_train['data'])

training_data = pd.DataFrame(countvect.transform(data_train['data']).todense(), 
                             columns = countvect.get_feature_names())
X_train = training_data
training_data.shape

(2034, 26879)

In [46]:
from sklearn.feature_extraction import stop_words
countvect_w_stops = CountVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)
countvect_w_stops.fit(data_train['data'])

training_data = pd.DataFrame(countvect_w_stops.transform(data_train['data']).todense(), 
                             columns = countvect_w_stops.get_feature_names())
training_data.shape

(2034, 26576)

In [60]:
print(training_data.sum(axis = 0).sort_values(ascending = False)[0:20])

space       1061
people       793
god          745
don          730
like         682
just         675
does         600
know         592
think        584
time         546
image        534
edu          501
use          468
good         449
data         444
nasa         419
graphics     414
jesus        411
say          409
way          387
dtype: int64


In [63]:
# countvect.fit(data_test['data'])
X_test = pd.DataFrame(countvect.transform(data_test['data']).todense(), 
                             columns = countvect.get_feature_names())
y_test = data_test['target']

In [74]:
from sklearn.linear_model import LogisticRegression

y_train = data_train['target']

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg_predictions = logreg.predict(X_test)
logreg.score(X_test, y_test)

0.71988174427198814

In [78]:
from sklearn.metrics import confusion_matrix, accuracy_score

con_matrix = confusion_matrix(y_test, logreg_predictions)
print(data_test.target_names)
print(con_matrix)

['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
[[185  26  41  67]
 [ 14 345  24   6]
 [ 26  48 307  13]
 [ 69  21  24 137]]


## 3. Hashing and TF-IDF

Let's see if Hashing or TF-IDF improves the accuracy.

1. Initialize a HashingVectorizer and repeat the test with no restriction on the number of features
- does the score improve with respect to the count vectorizer?
    - can you change any of the default parameters to improve it?
- print out the number of features for this model
- Initialize a TF-IDF Vectorizer and repeat the analysis above
- can you improve on your best score above?
    - can you change any of the default parameters to improve it?
- print out the number of features for this model

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer

## 4. Classifier comparison

Of all the vectorizers tested above, choose one that has a reasonable performance with a manageable number of features and compare the performance of these models:

- KNN
- Logistic Regression
- Decision Trees
- Support Vector Machine
- Random Forest
- Extra Trees

In order to speed up the calculation it's better to vectorize the data only once and then compare the models.

## Bonus: Other classifiers

Adapt the code from [this example](http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html#example-text-document-classification-20newsgroups-py) to compare across all the classifiers suggested and to display the final plot

## Bonus: NLTK

NLTK is a vast library. Can you find some interesting bits to share with classmates?
Start here: http://www.nltk.org/